In [61]:
import cv2
import os 
import numpy as np 
from sklearn.model_selection import train_test_split
from skimage.feature import hog
from tqdm import tqdm 

In [62]:
DATASET_PATH = './dataset/'
IMG_WIDTH = 224
IMG_HEIGHT = 224
BATCH_SIZE = 32

In [63]:
# For ORB
MAX_FEATURES = 100  # Number of max keypoint that will try
ORB_DESC_SIZE = 32      # ORB Descriptor Size (selalu 32 byte)

# For AKAZE
AKAZE_DESC_SIZE = 61

In [64]:
print('Getting file paths and labels')

image_paths = []
labels = []

positive_path = os.path.join(DATASET_PATH, 'Positive')
negative_path = os.path.join(DATASET_PATH, 'Negative')

for filename in os.listdir(positive_path):
    image_paths.append(os.path.join(positive_path, filename))
    labels.append(1)
    
for filename in os.listdir(negative_path):
    image_paths.append(os.path.join(negative_path, filename))
    labels.append(0)
    
image_paths = np.array(image_paths)
labels = np.array(labels)

X_train_paths, X_test_paths, y_train, y_test = train_test_split(
    image_paths, labels, test_size=0.25, random_state=42, stratify=labels
)

print(f'Total images: {len(image_paths)}')
print(f'Training test size: {len(X_train_paths)}')
print(f'Testing test size: {len(X_test_paths)}')

Getting file paths and labels
Total images: 40000
Training test size: 30000
Testing test size: 10000


#### HOG (Histogram of Oriented Gradients) Pipeline => Global Feature Descriptor


In [65]:
def feature_generator_hog(image_paths, labels, batch_size):
    num_samples = len(image_paths)
    
    while True:
        indices = np.arange(num_samples)

        # Dishuffle biar gak terjadi data leakage
        np.random.shuffle(indices)
        
        shuffled_paths = image_paths[indices]
        shuffled_labels = labels[indices]
        
        for i in range(0, num_samples, batch_size):
            batch_paths = shuffled_paths[i:i+batch_size]
            batch_labels = shuffled_labels[i:i+batch_size]
            
            batch_features = []
            
            for img_path in batch_paths:
                image = cv2.imread(img_path)
                image = cv2.resize(image, (IMG_WIDTH, IMG_HEIGHT)) # Ini wajib karena HOG menghasilkan vektor fitur dengan panjang yang bergantung pada ukuran gambar. Semua gambar harus berukuran sama agar fiturnya bisa dibandingkan.
                
                gray_image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
                
                features = hog( gray_image, 
                                # Fungsi dari pixel per cell karena di dalam setiap sel, HOG menghitung "arah" gradien (perubahan dari gelap ke terang) dan membuat histogram kecil. 
                                # Intinya, ini menangkap struktur lokal seperti tepi atau sudut dalam area 16x16 piksel.

                                pixels_per_cell=(16, 16), 
                                # Blok digunakan untuk normalisasi kontras. Ini membuat fitur HOG lebih tahan terhadap perubahan pencahayaan.
                                # Seperti contohnya retakan beton di bawah sinar matahari terik dan di bawah bayangan akan memiliki fitur HOG yang mirip karena adanya normalisasi ini.                  
                               
                               cells_per_block=(2, 2),
                               visualize=False,
                               block_norm='L2-Hys',
                               # Metode yang digunakan untuk normalisasi di dalam blok. 'L2-Hys' (L2-Hysteresis) adalah metode standar yang sangat populer dan bekerja dengan baik. 
                               # Ia melakukan normalisasi L2, membatasi nilai maksimum (clipping), lalu menormalisasi lagi.
                               
                               feature_vector=True
                               # Ini memastikan output dari fungsi hog adalah satu vektor 1D yang datar (flattened). 
                               # Jika False, outputnya akan berupa array multi-dimensi. 
                               # Untuk dimasukkan ke model klasifikasi standar (seperti SVM atau Random Forest), kita memerlukan vektor 1D.
                               ) 
                
                batch_features.append(features)
            
            yield np.array(batch_features), np.array(batch_labels)

#### ORB (Oriented FAST and Rotated BRIEF) Pipeline => Local Feature Descriptor


In [66]:
def feature_generator_orb(image_paths, labels, batch_size):
    num_samples = len(image_paths)
    
    orb = cv2.ORB_create(nfeatures=MAX_FEATURES)
    # Ini memberitahu ORB untuk mencari maksimal MAX_FEATURES (misalnya 100) titik paling "menarik" di gambar. 
    # Ini adalah langkah pertama untuk memastikan output kita konsisten.
    
    while True:
        indices = np.arange(num_samples)
        np.random.shuffle(indices)
        
        shuffled_paths = image_paths[indices]
        shuffled_labels = labels[indices]
        
        for i in range(0, num_samples, batch_size):
            batch_paths = shuffled_paths[i:i+batch_size]
            batch_labels = shuffled_labels[i:i+batch_size]
            
            batch_features = []
            
            for img_path in batch_paths:
                image = cv2.imread(img_path)
                image = cv2.resize(image, (IMG_WIDTH, IMG_HEIGHT))
                
                gray_image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
                
                kps, des = orb.detectAndCompute(gray_image, None)
                # kps => keypoints
                # des => descriptor
                
                flat_features = np.zeros(MAX_FEATURES * ORB_DESC_SIZE, dtype=np.float32)
                # awal kita buat semuanya itu menjadi [0,0]
                if des is not None:
                    num_descriptors = min(len(des), MAX_FEATURES) # capping => tidak ambil lebih dari batas feature jumlah maksimal
                    flat_features[:num_descriptors * ORB_DESC_SIZE] = des[:num_descriptors].ravel()
                    # des[:num_descriptors] mengambil semua deskriptor yang valid (maksimal 100).

                    # .ravel() "meratakan" array (100, 32) menjadi satu baris panjang (3200,).

                    # flat_features[...] = ... menyalin nilai-nilai ini ke dalam "wadah" nol yang kita siapkan.
                flat_features /= (np.linalg.norm(flat_features) + 1e-6)
                # flat_features /= ...: Vektor yang sudah jadi kemudian dinormalisasi. 
                # Ini membuat fitur lebih tahan terhadap perbedaan pencahayaan antar gambar.
                
                batch_features.append(flat_features)
            
            yield np.array(batch_features), np.array(batch_labels)

#### MSER (Maximally Stable Extermal Regions) => Global Feature Detector

In [67]:
def feature_generator_mser(image_paths, labels, batch_size):
    num_samples = len(image_paths)

    mser = cv2.MSER_create()
    
    while True:
        indices = np.arange(num_samples)
        np.random.shuffle(indices)
        
        shuffled_paths = image_paths[indices]
        shuffled_labels = labels[indices]
        
        for i in range(0, num_samples, batch_size):
            batch_paths = shuffled_paths[i:i+batch_size]
            batch_labels = shuffled_labels[i:i+batch_size]
            
            batch_features = []
            
            for img_path in batch_paths:
                image = cv2.imread(img_path)
                image = cv2.resize(image, (IMG_WIDTH, IMG_HEIGHT))
                
                gray_image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
                
                # Deteksi region MSER
                # MSER membutuhkan gambar 8-bit single-channel
                regions, _ = mser.detectRegions(gray_image)
                
                num_regions = len(regions)
                
                if num_regions > 0:
                    # Hitung properti statistik dari region
                    areas = [cv2.contourArea(r) for r in regions]
                    avg_area = np.mean(areas)
                    std_area = np.std(areas)
                    # Rasio total area region terhadap area gambar
                    total_area_ratio = np.sum(areas) / (IMG_WIDTH * IMG_HEIGHT)
                    
                    # Gabungkan menjadi satu feature vector
                    features = np.array([num_regions, avg_area, std_area, total_area_ratio])
                else:
                    # Jika tidak ada region yang terdeteksi, fiturnya nol semua
                    features = np.zeros(4, dtype=np.float32)

                batch_features.append(features)
            
            yield np.array(batch_features), np.array(batch_labels)

#### AKAZE (Accelerated-KAZE) Pipeline => Local Feature Detector

In [68]:
def feature_generator_akaze(image_paths, labels, batch_size):
    num_samples = len(image_paths)

    akaze = cv2.AKAZE_create()
    
    while True:
        indices = np.arange(num_samples)
        np.random.shuffle(indices)
        
        shuffled_paths = image_paths[indices]
        shuffled_labels = labels[indices]
        
        for i in range(0, num_samples, batch_size):
            batch_paths = shuffled_paths[i:i+batch_size]
            batch_labels = shuffled_labels[i:i+batch_size]
            
            batch_features = []
            
            for img_path in batch_paths:
                image = cv2.imread(img_path)
                image = cv2.resize(image, (IMG_WIDTH, IMG_HEIGHT))
                
                gray_image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

                kps, des = akaze.detectAndCompute(gray_image, None)

                flat_features = np.zeros(MAX_FEATURES * AKAZE_DESC_SIZE, dtype=np.float32)
                
                if des is not None:
                    num_descriptors = min(len(des), MAX_FEATURES)

                    flat_features[:num_descriptors * AKAZE_DESC_SIZE] = des[:num_descriptors].ravel()
                
                flat_features /= (np.linalg.norm(flat_features) + 1e-6)
                
                batch_features.append(flat_features)
            
            yield np.array(batch_features), np.array(batch_labels)

In [69]:
# Verification
print('Verification of HOG Pipeline')
train_gen_hog = feature_generator_hog(X_train_paths, y_train, BATCH_SIZE)

print('fetching one batch of feature vectors to test')
sample_batch_features_hog, sample_batch_labels_hog = next(train_gen_hog) 

print('pipeline complete, ready for training')
print(f'shape of one batch of features: {sample_batch_features_hog.shape}') # 32 per batch and 10 length
print(f'shape of one batch of labels: {sample_batch_labels_hog.shape}')  # 32 per batch
print(f'example feature vector (first image in batch:\n {sample_batch_features_hog[0]})') # 10 arrays

Verification of HOG Pipeline
fetching one batch of feature vectors to test
pipeline complete, ready for training
shape of one batch of features: (32, 6084)
shape of one batch of labels: (32,)
example feature vector (first image in batch:
 [0.22943039 0.07381583 0.22943039 ... 0.23797284 0.23797284 0.18398376])


In [70]:
# Verification
print('Verification of ORB Pipeline')
train_gen_orb = feature_generator_orb(X_train_paths, y_train, BATCH_SIZE)

print('fetching one batch of feature vectors to test')
sample_batch_features_orb, sample_batch_labels_orb = next(train_gen_orb) 

print('pipeline complete, ready for training')
print(f'shape of one batch of features: {sample_batch_features_orb.shape}') # 32 per batch and 10 length
print(f'shape of one batch of labels: {sample_batch_labels_orb.shape}')  # 32 per batch
print(f'example feature vector (first image in batch:\n {sample_batch_features_orb[0]})') # 10 arrays

Verification of ORB Pipeline
fetching one batch of feature vectors to test
pipeline complete, ready for training
shape of one batch of features: (32, 3200)
shape of one batch of labels: (32,)
example feature vector (first image in batch:
 [0.01405322 0.01565422 0.04073654 ... 0.         0.         0.        ])


In [71]:
# Verification
print('Verification of MSER Pipeline')
train_gen_mser = feature_generator_mser(X_train_paths, y_train, BATCH_SIZE)

print('fetching one batch of feature vectors to test')
sample_batch_features_mser, sample_batch_labels_mser = next(train_gen_mser) 

print('pipeline complete, ready for training')
print(f'shape of one batch of features: {sample_batch_features_mser.shape}') # 32 per batch and 10 length
print(f'shape of one batch of labels: {sample_batch_labels_mser.shape}')  # 32 per batch
print(f'example feature vector (first image in batch:\n {sample_batch_features_mser[0]})') # 10 arrays

Verification of MSER Pipeline
fetching one batch of feature vectors to test
pipeline complete, ready for training
shape of one batch of features: (32, 4)
shape of one batch of labels: (32,)
example feature vector (first image in batch:
 [3.60000000e+01 9.73923611e+03 1.00767035e+04 6.98765346e+00])


In [72]:
# Verification
print('Verification of AKAZE Pipeline')
train_gen_akaze = feature_generator_akaze(X_train_paths, y_train, BATCH_SIZE)

print('fetching one batch of feature vectors to test')
sample_batch_features_akaze, sample_batch_labels_akaze = next(train_gen_akaze) 

print('pipeline complete, ready for training')
print(f'shape of one batch of features: {sample_batch_features_akaze.shape}') # 32 per batch and 10 length
print(f'shape of one batch of labels: {sample_batch_labels_akaze.shape}')  # 32 per batch
print(f'example feature vector (first image in batch:\n {sample_batch_features_akaze[0]})') # 10 arrays

Verification of AKAZE Pipeline
fetching one batch of feature vectors to test


pipeline complete, ready for training
shape of one batch of features: (32, 6100)
shape of one batch of labels: (32,)
example feature vector (first image in batch:
 [0. 0. 0. ... 0. 0. 0.])
